<a href="https://colab.research.google.com/github/ssk-2610/ML_projects/blob/main/bosch_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
#libraries
from keras.datasets import cifar10
from keras.applications.vgg16 import VGG16
import matplotlib.pyplot as plt
import numpy as np
from keras.utils import np_utils
import cv2
from tensorflow.keras import layers,Model
from tensorflow.keras.models import Sequential
from keras.layers import Flatten,Dense
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
from keras.utils import to_categorical
IMAGE_SIZE=[32,32]


In [83]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

#separating data
#train
index1=np.where(y_train==0)
index2=np.where(y_train==1)
index3=np.where(y_train==9)
index=np.append(index1[0],index2[0])
index=np.append(index,index3[0])
X_train=X_train[index]
y_train=y_train[index]
for i in range(len(y_train)):
  if(y_train[i]==9):
    y_train[i]=2

#test
index1=np.where(y_test==0)
index2=np.where(y_test==1)
index3=np.where(y_test==9)
index=np.append(index1[0],index2[0])
index=np.append(index,index3[0])
X_test=X_test[index]
y_test=y_test[index]

for i in range(len(y_test)):
  if(y_test[i]==9):
    y_test[i]=2
    
X_train=X_train.astype('float32')
X_test=X_test.astype('float32')
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)




In [67]:
modelv =VGG16(input_shape=(32,32,3),weights='imagenet',include_top=False)


In [68]:
for layer in modelv.layers:
	layer.trainable=False

In [69]:
x = Flatten()(modelv.output)
x = Dense(3, activation = 'softmax')(x)
model = Model(inputs = modelv.input, outputs = x)

In [85]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0   

In [70]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [71]:
training_datagen = ImageDataGenerator(
                                    rescale=1./255,   # all pixel values will be between 0 an 1
                                    shear_range=0.2, 
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    preprocessing_function=preprocess_input)

validation_datagen = ImageDataGenerator(rescale = 1./255, 
                                        preprocessing_function=preprocess_input)

In [74]:
training_datagen.fit(X_train)
validation_datagen.fit(X_test)

In [80]:
history=model.fit(training_datagen.flow(X_train, y_train, batch_size=200),
          steps_per_epoch=len(X_train) / 100, epochs=20,validation_data=(X_test,y_test),validation_steps = 3000)

Epoch 1/20
75/75 [==============================] - 21s 277ms/step - loss: 0.4999 - accuracy: 0.7935 - val_loss: 26.9372 - val_accuracy: 0.6540
Epoch 2/20
75/75 [==============================] - 20s 268ms/step - loss: 0.4979 - accuracy: 0.7969 - val_loss: 27.1862 - val_accuracy: 0.6510
Epoch 3/20
75/75 [==============================] - 20s 269ms/step - loss: 0.5023 - accuracy: 0.7918 - val_loss: 27.2379 - val_accuracy: 0.6507
Epoch 4/20
75/75 [==============================] - 20s 272ms/step - loss: 0.5012 - accuracy: 0.7953 - val_loss: 27.3509 - val_accuracy: 0.6520
Epoch 5/20
75/75 [==============================] - 20s 267ms/step - loss: 0.5013 - accuracy: 0.7945 - val_loss: 27.7368 - val_accuracy: 0.6497
Epoch 6/20
75/75 [==============================] - 20s 272ms/step - loss: 0.4998 - accuracy: 0.7929 - val_loss: 27.5303 - val_accuracy: 0.6507
Epoch 7/20
75/75 [==============================] - 20s 269ms/step - loss: 0.4954 - accuracy: 0.7977 - val_loss: 27.6185 - val_accuracy:

In [89]:
print ('Training Accuracy = ' + str(history.history['accuracy'][-1])+" %")
print ('Validation Accuracy = ' + str(history.history['val_accuracy'][-1])+" %")
print('Validation loss = '+str(history.history['val_loss'][-1]))


Training Accuracy = 0.7979333400726318 %
Validation Accuracy = 0.6456666588783264 %
Validation loss = 30.164159774780273
